# ChatBot RAG com documentos PDF
---
Este notebook irá otimizar a classe para frontend e combina persistencia automatica + contextualização LCEL com recursos de múltiplos pdfs e merge de indices.

### IMPORTS

In [48]:
import os
import time
import shutil
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, Any, List, Optional, Union
from datetime import datetime
import json
import hashlib

# LLM 
from langchain_groq import ChatGroq

## core
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch

# pdf loader e vectordb
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS

# embeddings e textsplitters
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

### CONFIGURAÇÃO DA API

In [2]:
load_dotenv()

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY não foi configurada!")

print("API Key configurada!")

API Key configurada!


### CONFIGURAÇÕES GLOBAIS

In [92]:
GROQ_MODELS = {
    "llama-3.3-70b": "llama-3.3-70b-versatile",
    "llama-3.1-8b": "llama-3.1-8b-instant",
    "kimi-k2": "moonshotai/kimi-k2-instruct",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "qwen3-32b": "qwen/qwen3-32b"
}

DEFAULT_CONFIG = {
    "model": GROQ_MODELS["llama-3.1-8b"],
    "temperature": 0.1,
    "max_tokens": 2048,
    "chunk_size": 1000,
    "chunk_overlap": 200,
    "embeddings_model": "sentence-transformers/all-MiniLM-L6-v2",
    "retriever_k": 4,
    "retriever_fetch_k": 8,
    "max_history_messages": 10,
    "index_dir": "indices",
}

LLM_DEPENDENT = ['model', 'temperature', 'max_tokens']
RETRIEVER_DEPENDENT = ['retriever_k', 'retriever_fetch_k']
EMBEDDING_DEPENDENT = ['embeddings_model', 'chunk_size', 'chunk_overlap']

### CLASSE PRINCIPAL

In [79]:
class RAGChatBot:
    """
    ChatBot com Lazy Loading e Configuração Dinâmica.
    """

    def __init__(self, config: Dict[str, Any] = None, api_key: str = None):
        self.config = config or DEFAULT_CONFIG.copy()
        self.api_key = api_key or os.getenv('GROQ_API_KEY')

        if not self.api_key:
            raise ValueError('GROQ_API_KEY não configurada!')

        # componentes
        self.llm = None
        self.embeddings = None
        self.history: List[Any] = []

        # indices
        self.indices: Dict[str, FAISS] = {}
        self.retrievers: Dict[str, Any] = {}

        print('\nInicializando ChatBot...')
        self._init_llm()
        self._init_embeddings()
        self._init_system_prompt()
        print('ChatBot pronto!\n')

    def _init_llm(self):
        """Inicializa LLM."""
        self.llm = ChatGroq(
            model=self.config['model'],
            temperature=self.config['temperature'],
            max_tokens=self.config['max_tokens'],
            api_key=self.api_key
        )
        print(f'LLM: {self.config["model"]} (temp={self.config["temperature"]}, tokens={self.config["max_tokens"]})')

    def _init_embeddings(self):
        """Inicializa embeddings."""
        print(f"Embeddings: {self.config['embeddings_model']}")
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.config['embeddings_model'],
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': True},
        )

    def _init_system_prompt(self):
        """System prompt."""
        system_msg = SystemMessage(
            content="""Você é um assistente especializado em responder com base em documentos fornecidos.""")
        self.history.append(system_msg)

    # ========================================================================
    # UPDATE CONFIG -> PARA FRONTEND
    # ========================================================================

    def update_config(self, **kwargs) -> Dict[str, Any]:
        """
        Atualiza configurações dinamicamente. Recria componentes afetados.

        Args:
            **kwargs: Configs a atualizar (model, temperature, max_tokens, 
                     retriever_k, retriever_fetch_k, etc.)

        Returns:
            Dict com success, message, changes, recreated, current_config
        """
        result = {
            'success': False,
            'message': '',
            'changes': [],
            'recreated': [],
            'current_config': {},
            'warning': None
        }

        # validar chaves
        valid_keys = set(DEFAULT_CONFIG.keys())
        invalid = [k for k in kwargs if k not in valid_keys]
        if invalid:
            result['message'] = f"Inválidas: {', '.join(invalid)}"
            result['current_config'] = self.get_config()
            return result

        # validar valores
        errors = []
        if 'temperature' in kwargs:
            t = kwargs['temperature']
            if not isinstance(t, (int, float)) or not (0.0 <= t <= 2.0):
                errors.append("temperature: 0.0-2.0")

        if 'max_tokens' in kwargs:
            if not isinstance(kwargs['max_tokens'], int) or kwargs['max_tokens'] < 1:
                errors.append("max_tokens: inteiro > 0")

        if 'retriever_k' in kwargs:
            if not isinstance(kwargs['retriever_k'], int) or kwargs['retriever_k'] < 1:
                errors.append("retriever_k: inteiro > 0")

        if errors:
            result['message'] = "Erros: " + "; ".join(errors)
            result['current_config'] = self.get_config()
            return result

        # detectar mudanças
        changes = []
        needs_llm = False
        needs_retriever = False
        needs_embedding = False

        for key, new_val in kwargs.items():
            old_val = self.config.get(key)
            if old_val != new_val:
                self.config[key] = new_val
                changes.append(f"{key}: {old_val} → {new_val}")

                if key in LLM_DEPENDENT:
                    needs_llm = True
                if key in RETRIEVER_DEPENDENT:
                    needs_retriever = True
                if key in EMBEDDING_DEPENDENT:
                    needs_embedding = True

        if not changes:
            result['success'] = True
            result['message'] = "Nenhuma alteração"
            result['current_config'] = self.get_config()
            return result

        # aplicar mudanças
        try:
            if needs_embedding:
                print("\nRecriando embeddings...")
                self._init_embeddings()
                result['recreated'].append('embeddings')
                # embeddings mudou → índices recriados
                if self.indices:
                    print("Modelo de embeddings mudou! Índices precisam ser recarregados.")
                    self.indices = {}
                    self.retrievers = {}
                    result['warning'] = "Recarregue os documentos (embeddings alterados)"

            if needs_llm:
                print("Recriando LLM...")
                self._init_llm()
                result['recreated'].append('llm')

            if needs_retriever and self.indices:
                print("Recriando retrievers...")
                self._recreate_retrievers()
                result['recreated'].append('retrievers')

            # recriar chain se LLM ou retriever mudou
            if (needs_llm or needs_retriever) and self.indices:
                print("Recriando RAG chain...")
                self._create_unified_rag_chain()
                result['recreated'].append('rag_chain')

            result['success'] = True
            result['message'] = f"Atualizado ({len(changes)} mudanças)"
            result['changes'] = changes

        except Exception as e:
            result['message'] = f"Erro: {str(e)}"

        result['current_config'] = self.get_config()
        return result

    def _recreate_retrievers(self):
        """Recria todos os retrievers com novos parâmetros."""
        self.retrievers = {}
        for name, vectorstore in self.indices.items():
            self.retrievers[name] = vectorstore.as_retriever(
                search_type="mmr",
                search_kwargs={
                    "k": self.config["retriever_k"],
                    "fetch_k": self.config["retriever_fetch_k"]
                }
            )

    def get_config(self) -> Dict[str, Any]:
        """Retorna config atual."""
        return self.config.copy()

    def reset_config(self) -> Dict[str, Any]:
        """Reseta para padrão."""
        # preservar index_dir e outras configs não-LLM
        preserved = {k: v for k, v in self.config.items() 
                    if k not in DEFAULT_CONFIG or k in ['index_dir']}
        default = DEFAULT_CONFIG.copy()
        default.update(preserved)
        return self.update_config(**default)

    # ========================================================================
    # CARGA PDFs
    # ========================================================================

    def load_documents(self, pdf_paths: Union[str, List[str]]) -> Dict[str, Any]:
        """
        Carrega lista de PDFs com lazy loading.
        Para cada PDF: se índice existe carrega, senão cria.
        """
        if isinstance(pdf_paths, str):
            pdf_paths = [pdf_paths]

        results = {'loaded': [], 'created': [], 'failed': [], 'total': 0}

        print(f"\nProcessando {len(pdf_paths)} documento(s)...")
        print("=" * 60)

        for pdf_path in pdf_paths:
            pdf_path = os.path.abspath(pdf_path)

            if not os.path.exists(pdf_path):
                print(f"Não encontrado: {pdf_path}")
                results['failed'].append({'file': pdf_path, 'error': 'Not found'})
                continue

            pdf_name = Path(pdf_path).stem
            safe_name = self._sanitize_name(pdf_name)
            index_path = os.path.join(self.config['index_dir'], safe_name)

            print(f"\n{pdf_name}")

            try:
                if os.path.exists(index_path) and os.path.isdir(index_path):
                    # CARREGAR DO DISCO
                    print(f"Carregando índice existente...")
                    self._load_single_index(safe_name, index_path)
                    results['loaded'].append(pdf_name)
                    print(f"Carregado")
                else:
                    # CRIAR NOVO
                    print(f"   🔧 Criando novo índice...")
                    success = self._create_single_index(pdf_path, safe_name, index_path)
                    if success:
                        results['created'].append(pdf_name)
                        print(f"Criado")
                    else:
                        results['failed'].append({'file': pdf_name, 'error': 'Process failed'})

            except Exception as e:
                print(f"Erro: {str(e)}")
                results['failed'].append({'file': pdf_name, 'error': str(e)})

        results['total'] = len(self.indices)

        print("\n" + "=" * 60)
        print(f"Resumo: {len(results['loaded'])} carregados, {len(results['created'])} criados, {len(results['failed'])} falhas")
        print("=" * 60)

        if self.indices:
            self._create_unified_rag_chain()

        return results

    def _sanitize_name(self, name: str) -> str:
        """Nome seguro para pasta."""
        safe = "".join(c if c.isalnum() or c in ('-', '_') else '_' for c in name)
        return safe[:50]

    def _create_single_index(self, pdf_path: str, name: str, index_path: str) -> bool:
        """Cria índice para um PDF."""
        os.makedirs(self.config['index_dir'], exist_ok=True)

        loader = PyMuPDFLoader(pdf_path)
        docs = loader.load()

        if not docs:
            return False

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.config["chunk_size"],
            chunk_overlap=self.config["chunk_overlap"],
            separators=["\n\n", "\n", ".", "!", "?", ";", " ", ""]
        )
        chunks = text_splitter.split_documents(docs)

        if not chunks:
            return False

        for chunk in chunks:
            chunk.metadata['source_file'] = Path(pdf_path).name
            chunk.metadata['source_name'] = name

        vectorstore = FAISS.from_documents(chunks, self.embeddings)
        vectorstore.save_local(index_path)

        # metadados
        metadata = {
            'original_file': pdf_path,
            'name': name,
            'chunks': len(chunks),
            'pages': len(docs),
            'created_at': datetime.now().isoformat(),
            'config': {
                'embeddings_model': self.config['embeddings_model'],
                'chunk_size': self.config['chunk_size']
            }
        }
        with open(os.path.join(index_path, 'info.json'), 'w') as f:
            json.dump(metadata, f, indent=2)

        self.indices[name] = vectorstore
        self.retrievers[name] = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={"k": self.config["retriever_k"], "fetch_k": self.config["retriever_fetch_k"]}
        )

        return True

    def _load_single_index(self, name: str, index_path: str):
        """Carrega índice existente."""
        vectorstore = FAISS.load_local(
            index_path,
            self.embeddings,
            allow_dangerous_deserialization=True
        )

        self.indices[name] = vectorstore
        self.retrievers[name] = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={"k": self.config["retriever_k"], "fetch_k": self.config["retriever_fetch_k"]}
        )

    # ========================================================================
    # RAG CHAIN
    # ========================================================================

    def _search_all_indices(self, query: str) -> List[Document]:
        """Busca em TODOS os índices."""
        all_docs = []
        for name, retriever in self.retrievers.items():
            docs = retriever.invoke(query)
            for doc in docs:
                doc.metadata['index_source'] = name
            all_docs.extend(docs)
        return all_docs

    def _create_unified_rag_chain(self):
        """Cria chain que busca em todos os índices."""

        contextualize_q_prompt = ChatPromptTemplate.from_messages([
            ("system", "Reformule considerando histórico. Mantenha português."),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ])

        contextualize_chain = contextualize_q_prompt | self.llm | StrOutputParser()

        def get_context(x: dict) -> str:
            if x.get("chat_history") and len(x["chat_history"]) > 1:
                query = contextualize_chain.invoke(x)
            else:
                query = x["input"]

            docs = self._search_all_indices(query)

            formatted = []
            for i, doc in enumerate(docs, 1):
                page = doc.metadata.get("page", "?")
                source = doc.metadata.get("source_file", "?")
                formatted.append(f"[Doc {i} | {source} | Pág. {page}]\n{doc.page_content}")

            return "\n\n---\n\n".join(formatted)

        qa_prompt = ChatPromptTemplate.from_messages([
            ("system", """Responda baseado no contexto (múltiplos documentos).
Se não souber, diga "Não encontrei".
Cite a fonte.

Contexto:
{context}"""),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ])

        self.rag_chain = (
            RunnablePassthrough.assign(context=lambda x: get_context(x))
            | qa_prompt
            | self.llm
            | StrOutputParser()
        )

    # ========================================================================
    # CHAT
    # ========================================================================

    def chat(self, user_input: str, verbose: bool = True) -> Dict[str, Any]:
        """Processa pergunta."""
        start_time = time.time()

        if verbose:
            print(f"\n👤 {user_input}")

        if not self.indices:
            # Modo chat puro
            self.history.append(HumanMessage(content=user_input))
            response = self.llm.invoke(self.history)
            self.history.append(AIMessage(content=response.content))

            if verbose:
                print(f"\n🤖 {response.content}")

            return {"answer": response.content, "mode": "CHAT", "sources": []}

        # Modo RAG
        if verbose:
            print(f"🔍 Buscando em {len(self.indices)} documento(s)...")

        response = self.rag_chain.invoke({
            "input": user_input,
            "chat_history": self.history
        })

        sources = self._search_all_indices(user_input)

        self.history.append(HumanMessage(content=user_input))
        self.history.append(AIMessage(content=response))

        elapsed = time.time() - start_time

        if verbose:
            print(f"\n🤖 {response}")
            print(f"\n📚 Fontes: {len(sources)}")
            by_source = {}
            for doc in sources[:6]:
                src = doc.metadata.get('source_file', '?')
                by_source.setdefault(src, []).append(doc.metadata.get('page', '?'))
            for src, pages in by_source.items():
                print(f"   📄 {src} (págs: {', '.join(map(str, pages[:3]))})")
            print(f"\n⏱️  {elapsed:.2f}s")

        self._trim_history()

        return {"answer": response, "mode": "RAG", "sources": sources, "time": elapsed}

    def _trim_history(self):
        """Limita histórico."""
        max_msgs = self.config["max_history_messages"] * 2
        if len(self.history) > max_msgs + 1:
            self.history = [self.history[0]] + self.history[-max_msgs:]

    # ========================================================================
    # UTILITÁRIOS
    # ========================================================================

    def list_loaded_documents(self) -> List[Dict[str, Any]]:
        """Lista documentos carregados."""
        docs = []
        for name in self.indices.keys():
            index_path = os.path.join(self.config['index_dir'], name)
            info_path = os.path.join(index_path, 'info.json')

            info = {}
            if os.path.exists(info_path):
                with open(info_path, 'r') as f:
                    info = json.load(f)

            docs.append({
                'name': name,
                'original_file': info.get('original_file', '?'),
                'chunks': info.get('chunks', 0),
                'pages': info.get('pages', 0),
                'created': info.get('created_at', '?')
            })
        return docs

    def remove_document(self, name: str) -> bool:
        """Remove documento da memória."""
        if name not in self.indices:
            print(f"'{name}' não carregado")
            return False

        del self.indices[name]
        del self.retrievers[name]

        if self.indices:
            self._create_unified_rag_chain()
        else:
            self.rag_chain = None

        print(f"'{name}' removido da memória")
        return True

    def clear_all(self):
        """Remove todos da memória."""
        self.indices = {}
        self.retrievers = {}
        self.rag_chain = None
        print("Memória limpa")

    def delete_index(self, name: str) -> bool:
        """Deleta índice do disco permanentemente."""
        index_path = os.path.join(self.config['index_dir'], name)

        if not os.path.exists(index_path):
            print(f"Índice '{name}' não existe")
            return False

        # remover da memória se estiver carregado
        if name in self.indices:
            self.remove_document(name)

        # deletar do disco
        shutil.rmtree(index_path)
        print(f"Índice '{name}' deletado do disco")
        return True

    def get_stats(self) -> Dict[str, Any]:
        """Estatísticas."""
        return {
            'documents_in_memory': len(self.indices),
            'document_names': list(self.indices.keys()),
            'history_messages': len(self.history),
            'model': self.config['model'],
            'temperature': self.config['temperature'],
            'embeddings': self.config['embeddings_model']
        }


### INICIALIZAR CHATBOT

In [82]:
bot = RAGChatBot()


🚀 Inicializando RAGChatBot Pro...
🤖 LLM: llama-3.1-8b-instant (temp=0.1, tokens=2048)
📊 Embeddings: sentence-transformers/all-MiniLM-L6-v2
✅ Bot pronto!



In [67]:
bot.get_stats()

{'documents_in_memory': 0,
 'document_names': [],
 'history_messages': 1,
 'model': 'llama-3.1-8b-instant',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

### FAZER PERGUNTAS

#### CHAT

In [68]:
bot.chat('Qual a capital da França');


👤 Qual a capital da França

🤖 A capital da França é Paris.


In [70]:
bot.chat('Resuma econometria em uma frase curta e provocante.');


👤 Resuma econometria em uma frase curta e provocante.

🤖 "A econometria é a arte de encontrar padrões ocultos em números, mas nem sempre é fácil descobrir o que eles estão tentando nos dizer."


In [71]:
bot.chat('Explique machine learning em uma linha e somente com emojis',verbose=False)

{'answer': '🤖💻📊📈💡🔍', 'mode': 'CHAT', 'sources': []}

In [72]:
res = bot.chat('qual foi minha primeira pergunta?', verbose=False)
res['answer']

'Sua primeira pergunta foi sobre a capital da França.'

#### CHAT COM RAG

In [74]:
bot.get_stats()

{'documents_in_memory': 0,
 'document_names': [],
 'history_messages': 9,
 'model': 'llama-3.1-8b-instant',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

##### CARREGANDO UM PDF

In [83]:
bot.load_documents("C:\\Users\\wsant\\Downloads\\O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf")


📚 Processando 1 documento(s)...

📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao
   🔧 Criando novo índice...
   ✅ Criado

📊 Resumo: 0 carregados, 1 criados, 0 falhas


{'loaded': [],
 'created': ["O'Reilly - PT - SQL Guia Prático - Alice Zhao"],
 'failed': [],
 'total': 1}

In [84]:
bot.chat('Qual o principal tema deste documento?');


👤 Qual o principal tema deste documento?
🔍 Buscando em 1 documento(s)...

🤖 O principal tema deste documento é SQL (Structured Query Language), especificamente um guia prático para aprender e trabalhar com SQL.

📚 Fontes: 4
   📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf (págs: 25, 2, 13)

⏱️  0.54s


In [85]:
bot.chat("Explique para um leigo em um parágrafo sobre o comando SELECT");


👤 Explique para um leigo em um parágrafo sobre o comando SELECT
🔍 Buscando em 1 documento(s)...

🤖 O comando SELECT é um dos principais comandos do SQL e serve para selecionar os dados que você deseja ver de uma tabela de banco de dados. Imagine que você está procurando por uma informação específica em um grande livro de notas. O comando SELECT é como uma lupa que permite você escolher apenas as informações que você precisa, como o nome, a idade ou a nota de um aluno, por exemplo. Com o SELECT, você pode especificar quais colunas (campos) da tabela você deseja ver e, em seguida, o SQL mostrará apenas essas informações, tornando mais fácil encontrar o que você está procurando.

📚 Fontes: 4
   📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf (págs: 59, 78, 83)

⏱️  1.23s


In [86]:
bot.get_stats()

{'documents_in_memory': 1,
 'document_names': ['O_Reilly_-_PT_-_SQL_Guia_Pra_tico_-_Alice_Zhao'],
 'history_messages': 5,
 'model': 'llama-3.1-8b-instant',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

In [57]:
bot.clear_documents()

Documentos removidos!


##### CARREGANDO MULTIPLOS PDFs

In [87]:
pdfs = [
    "C:\\Users\\wsant\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf",
    "C:\\Users\\wsant\\Downloads\\000906000101011.pdf",
    "C:\\Users\\wsant\\Downloads\\O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf"
]

In [88]:
bot.load_documents(pdfs)


📚 Processando 3 documento(s)...

📄 NIPS-2017-attention-is-all-you-need-Paper
   🔧 Criando novo índice...
   ✅ Criado

📄 000906000101011
   🔧 Criando novo índice...
   ✅ Criado

📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao
   💾 Carregando índice existente...
   ✅ Carregado

📊 Resumo: 1 carregados, 2 criados, 0 falhas


{'loaded': ["O'Reilly - PT - SQL Guia Prático - Alice Zhao"],
 'created': ['NIPS-2017-attention-is-all-you-need-Paper', '000906000101011'],
 'failed': [],
 'total': 3}

In [89]:
bot.get_stats()

{'documents_in_memory': 3,
 'document_names': ['O_Reilly_-_PT_-_SQL_Guia_Pra_tico_-_Alice_Zhao',
  'NIPS-2017-attention-is-all-you-need-Paper',
  '000906000101011'],
 'history_messages': 5,
 'model': 'llama-3.1-8b-instant',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

In [90]:
bot.chat('Sobre o que são estes documentos?');


👤 Sobre o que são estes documentos?
🔍 Buscando em 3 documento(s)...

🤖 Os documentos fornecidos parecem ser uma coleção de textos relacionados a diferentes tópicos, incluindo:

1. Um guia prático de SQL (Structured Query Language) para aprender e trabalhar com banco de dados.
2. Um artigo sobre a arquitetura de rede neural chamada "Attention is All You Need" (Atenção é tudo o que você precisa), que foi apresentado em uma conferência de inteligência artificial.
3. Relatórios de empresa sobre a Bradesco, uma instituição financeira brasileira, que incluem informações sobre a estratégia de negócios, produtos e serviços oferecidos pela empresa.
4. Outros textos que não foram identificados com certeza, mas que parecem ser relacionados a tecnologia e negócios.

Em resumo, os documentos parecem ser uma mistura de conteúdo técnico e empresarial.

📚 Fontes: 12
   📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf (págs: 25, 178, 17)
   📄 NIPS-2017-attention-is-all-you-need-Paper.pdf (págs: 9,

#### ALTERANDO MODELO

In [93]:
bot.update_config(model=GROQ_MODELS['gpt-oss-20b'])

{'success': True,
 'message': 'Nenhuma alteração',
 'changes': [],
 'recreated': [],
 'current_config': {'model': 'openai/gpt-oss-20b',
  'temperature': 0.1,
  'max_tokens': 2048,
  'chunk_size': 1000,
  'chunk_overlap': 200,
  'embeddings_model': 'sentence-transformers/all-MiniLM-L6-v2',
  'retriever_k': 4,
  'retriever_fetch_k': 8,
  'max_history_messages': 10,
  'index_dir': 'indices'},
 'warning': None}

In [94]:
bot.get_stats()

{'documents_in_memory': 3,
 'document_names': ['O_Reilly_-_PT_-_SQL_Guia_Pra_tico_-_Alice_Zhao',
  'NIPS-2017-attention-is-all-you-need-Paper',
  '000906000101011'],
 'history_messages': 7,
 'model': 'openai/gpt-oss-20b',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

#### CHAT COM NOVO MODELO

In [95]:
bot.chat('Resuma os temas mais relavantes do bradesco. ');


👤 Resuma os temas mais relavantes do bradesco. 
🔍 Buscando em 3 documento(s)...

🤖 Com base nos documentos fornecidos, os temas mais relevantes do Bradesco são:

1. **Expansão de produtos e serviços**: O Bradesco está expandindo sua oferta de produtos e serviços, incluindo financiamentos, seguros, consórcios e soluções de pagamento.
2. **Inovação e tecnologia**: A empresa está investindo em tecnologia para melhorar a eficiência e a experiência do cliente, incluindo a implementação do Salesforce e a criação de um portal de desenvolvedores.
3. **Crescimento no mercado**: O Bradesco está crescendo no mercado, com um aumento de 2,3% no market share de seguros e um aumento de 3,5% no market share de DCM (Debt Capital Markets) em 2025.
4. **Expansão de canais de distribuição**: A empresa está expandindo seus canais de distribuição, incluindo a criação de um novo canal transacional web para pessoas jurídicas e a expansão de sua presença em plataformas de pagamento.
5. **Foco em SMEs**: O Bra

In [96]:
bot.chat('resuma o mecanismo de atenção em emojis.');


👤 resuma o mecanismo de atenção em emojis.
🔍 Buscando em 3 documento(s)...

🤖 🤔💡📝👀💬🔍️💻️

Essas emojis resumem o mecanismo de atenção, que é um componente fundamental da rede neural chamada "Attention is All You Need". Aqui está uma explicação de cada emoji:

🤔: Pensamento e processamento de informações
💡: Iluminação e compreensão da informação
📝: Processamento de linguagem natural
👀: Foco e atenção
💬: Comunicação e interação entre as partes da rede neural
🔍️: Busca e seleção de informações relevantes
💻️: Processamento e armazenamento de informações

📚 Fontes: 12
   📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf (págs: 157, 136, 183)
   📄 NIPS-2017-attention-is-all-you-need-Paper.pdf (págs: 1, 10)

⏱️  1.87s


In [97]:
bot.chat('me mostre um exemplo de query sql.');


👤 me mostre um exemplo de query sql.
🔍 Buscando em 3 documento(s)...

🤖 Aqui está um exemplo de query SQL simples:

```sql
SELECT nome, idade
FROM clientes
WHERE cidade = 'São Paulo';
```

Essa query faz o seguinte:

* `SELECT nome, idade`: Seleciona as colunas `nome` e `idade` da tabela `clientes`.
* `FROM clientes`: Indica a tabela que será consultada, que é a tabela `clientes`.
* `WHERE cidade = 'São Paulo'`: Filtro que seleciona apenas os registros onde a coluna `cidade` tem o valor `'São Paulo'`.

Essa query retornará uma lista com os nomes e idades dos clientes que residem em São Paulo.

📚 Fontes: 12
   📄 O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf (págs: 59, 1, 48)
   📄 NIPS-2017-attention-is-all-you-need-Paper.pdf (págs: 8, 2)

⏱️  1.47s


#### UTILITARIOS

In [99]:
bot.get_stats()

{'documents_in_memory': 3,
 'document_names': ['O_Reilly_-_PT_-_SQL_Guia_Pra_tico_-_Alice_Zhao',
  'NIPS-2017-attention-is-all-you-need-Paper',
  '000906000101011'],
 'history_messages': 13,
 'model': 'openai/gpt-oss-20b',
 'temperature': 0.1,
 'embeddings': 'sentence-transformers/all-MiniLM-L6-v2'}

In [101]:
bot.get_config()

{'model': 'openai/gpt-oss-20b',
 'temperature': 0.1,
 'max_tokens': 2048,
 'chunk_size': 1000,
 'chunk_overlap': 200,
 'embeddings_model': 'sentence-transformers/all-MiniLM-L6-v2',
 'retriever_k': 4,
 'retriever_fetch_k': 8,
 'max_history_messages': 10,
 'index_dir': 'indices'}

In [100]:
bot.list_loaded_documents()

[{'name': 'O_Reilly_-_PT_-_SQL_Guia_Pra_tico_-_Alice_Zhao',
  'original_file': "C:\\Users\\wsant\\Downloads\\O'Reilly - PT - SQL Guia Prático - Alice Zhao.pdf",
  'chunks': 537,
  'pages': 306,
  'created': '2026-02-09T00:26:28.974202'},
 {'name': 'NIPS-2017-attention-is-all-you-need-Paper',
  'original_file': 'C:\\Users\\wsant\\Downloads\\NIPS-2017-attention-is-all-you-need-Paper.pdf',
  'chunks': 43,
  'pages': 11,
  'created': '2026-02-09T00:27:50.841445'},
 {'name': '000906000101011',
  'original_file': 'C:\\Users\\wsant\\Downloads\\000906000101011.pdf',
  'chunks': 30,
  'pages': 25,
  'created': '2026-02-09T00:27:51.908349'}]

In [103]:
bot.update_config(retriever_k=3, retriever_fetch_k=5)

🔄 Recriando retrievers...
🔄 Recriando RAG chain...


{'success': True,
 'message': 'Atualizado (2 mudanças)',
 'changes': ['retriever_k: 4 → 3', 'retriever_fetch_k: 8 → 5'],
 'recreated': ['retrievers', 'rag_chain'],
 'current_config': {'model': 'openai/gpt-oss-20b',
  'temperature': 0.1,
  'max_tokens': 2048,
  'chunk_size': 1000,
  'chunk_overlap': 200,
  'embeddings_model': 'sentence-transformers/all-MiniLM-L6-v2',
  'retriever_k': 3,
  'retriever_fetch_k': 5,
  'max_history_messages': 10,
  'index_dir': 'indices'},
 'warning': None}

In [104]:
bot.get_config()

{'model': 'openai/gpt-oss-20b',
 'temperature': 0.1,
 'max_tokens': 2048,
 'chunk_size': 1000,
 'chunk_overlap': 200,
 'embeddings_model': 'sentence-transformers/all-MiniLM-L6-v2',
 'retriever_k': 3,
 'retriever_fetch_k': 5,
 'max_history_messages': 10,
 'index_dir': 'indices'}

In [105]:
bot.chat('o que é o foco em SMEs?');


👤 o que é o foco em SMEs?
🔍 Buscando em 3 documento(s)...

🤖 O foco em SMEs (Pequenas e Médias Empresas) é uma estratégia de negócios que visa atender às necessidades específicas dessas empresas.

SMEs são empresas que têm um número limitado de funcionários e um volume de negócios menor em comparação com as grandes empresas. Elas precisam de soluções de financiamento, seguros, tecnologia e outros produtos e serviços que sejam personalizados para suas necessidades específicas.

O foco em SMEs pode incluir:

* Oferecer soluções de financiamento personalizadas para ajudar as SMEs a crescerem e se desenvolverem.
* Desenvolver produtos e serviços de seguros que sejam adequados às necessidades das SMEs.
* Criar plataformas de tecnologia que sejam fáceis de usar e acessíveis para as SMEs.
* Oferecer consultoria e apoio para ajudar as SMEs a melhorar sua gestão e produtividade.

O objetivo do foco em SMEs é ajudar essas empresas a crescerem e se desenvolverem, criando empregos e contribuindo 